# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"jianches","key":"c1bda8e22ca7a63a4fe3cd4bc80e5823"}')
#     # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# # commands to download data from kaggle
# !kaggle competitions download -c 11785-hw1p2-s24

# !unzip -qo /content/11785-hw1p2-s24.zip -d '/content'

In [ ]:
# !rm -rf /content/11785-s24-hw1p2


In [ ]:
#print(root)  /content/11-785-s24-hw1p2/train-clean-100/mfcc/103-1240-0000.npy

In [ ]:
np.load('/content/11-785-s24-hw1p2/train-clean-100/mfcc/103-1240-0000.npy')

array([[ 1.10716181e+01,  1.29364958e+01,  2.11114330e+01, ...,
        -1.52556181e-01,  3.08246484e-13,  9.49062500e+01],
       [ 1.14747839e+01,  1.38595648e+01,  2.18050823e+01, ...,
        -2.66239017e-01,  3.11884416e-13,  9.56741714e+01],
       [ 1.12015600e+01,  1.66964321e+01,  2.29666538e+01, ...,
        -5.54687679e-01,  3.03849421e-13,  9.54803467e+01],
       ...,
       [ 2.51545238e+00, -1.28244057e+01,  2.81143970e+01, ...,
        -1.46406519e+00,  4.07646790e-13,  1.16040901e+02],
       [-3.72080021e-02, -1.21763525e+01,  2.60421238e+01, ...,
        -9.37022150e-01,  4.09246665e-13,  1.17274742e+02],
       [-1.40476334e+00, -1.17058363e+01,  2.43025761e+01, ...,
        -1.07578576e+00,  4.07066769e-13,  1.18305443e+02]], dtype=float32)

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100/", limit = None): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root,partition,'mfcc')
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(root,partition,'transcript')

        if limit is None:
            mfcc_names = sorted(os.listdir(self.mfcc_dir))
            transcript_names = sorted(os.listdir(self.transcript_dir))
        else:
            mfcc_names = sorted(os.listdir(self.mfcc_dir))[:limit]
            transcript_names = sorted(os.listdir(self.transcript_dir))[:limit]


        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        # total_timesteps = 0
        # # store the length of time step
        # for mfcc_file in mfcc_names:
        #   mfcc_path = os.path.join(self.mfcc_dir, mfcc_file)
        #   mfcc = np.load(mfcc_path)
        #   total_timesteps += mfcc.shape[0]

        # mfcc_placeholder = np.zeros((total_timesteps, 27), dtype=np.float32)  #
        # transcript_placeholder = np.zeros((total_timesteps,), dtype=np.uint8)  #

        self.mfccs, self.transcripts = [], []

        # current_index = 0
        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir,mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
        #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir,transcript_names[i])) # Remove [SOS] and [EOS] from the transcript
            transcript = transcript[1:-1]
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
            #num_timesteps = mfcc.shape[0]

            # mfcc_placeholder[current_index : num_timesteps + current_index] = mfcc
            # transcript_placeholder[current_index : num_timesteps + current_index] = transcript

            #current_index += mfcc.shape[0]
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        self.mfccs          = np.concatenate(self.mfccs,axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts,axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is.
        # We can introduce context by padding zeros on top and bottom of self.mfcc

        # zero_padding_matrix = np.zeros((self.mfccs.shape[0]+2*context,self.mfccs.shape[1]))
        # zero_padding_matrix[0+context:-context,:] = self.mfccs# TODO
        self.mfccs = np.pad(self.mfccs, pad_width = ((context, context), (0,0)), mode = 'constant')

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        phonemes_map = {phoneme: index for index, phoneme in enumerate(self.phonemes)}
        for i, elements in enumerate(self.transcripts):
          self.transcripts[i] = phonemes_map[elements]

        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind:ind+2*self.context+1,:]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.reshape(-1) # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(int(self.transcripts[ind]))

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
   # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "test-clean", limit = None):
        super().__init__()
        self.context    = context
        self.phonemes   = phonemes

        self.mfcc_dir = os.path.join(root, partition,'mfcc')

        if limit is None:
            mfcc_names = sorted(os.listdir(self.mfcc_dir))
        else:
            mfcc_names = sorted(os.listdir(self.mfcc_dir))[:limit]

        # total_timesteps = 0
        # # store the length of time step
        # for mfcc_file in mfcc_names:
        #   mfcc_path = os.path.join(self.mfcc_dir, mfcc_file)
        #   mfcc = np.load(mfcc_path)
        #   total_timesteps += mfcc.shape[0]

        #mfcc_placeholder = np.zeros((total_timesteps, 27), dtype=np.float32)  #

        self.mfccs = []
        #current_index = 0
        for i in range(len(mfcc_names)):
          mfcc = np.load(os.path.join(self.mfcc_dir,mfcc_names[i]))
          mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
          # num_step = mfcc.shape[0]
          # mfcc_placeholder[current_index : current_index + num_step] = mfcc
          # current_index += num_step
          self.mfccs.append(mfcc)

        self.mfccs = np.concatenate(self.mfccs, axis=0)


        self.length = len(self.mfccs)

        self.mfccs = np.pad(self.mfccs, pad_width = ((context, context), (0,0)), mode = 'constant')


    def __len__(self):
      return self.length

    def __getitem__(self, ind):

      frames = self.mfccs[ind:ind+2*self.context+1, :]
      frames = frames.reshape(-1)

      frames = torch.FloatTensor(frames)

      return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [ ]:
config = {
    'epochs'        : 30,
    'batch_size'    : 2048,
    'context'       : 50,
    'init_lr'       : 1e-5/2,
    'architecture'  : 'a-model',
    'dropout'    : 0.2
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:
#TODO: Create a dataset object using the AudioDataset class for the training data
x=np.load('/content/11-785-s24-hw1p2/train-clean-100/transcript/103-1240-0000.npy')
print(x.shape)
train_data = AudioDataset(root = "/content/11-785-s24-hw1p2/", context=config["context"], limit = None)
#
i,j = train_data[8]
print(i,j,i.shape,j.shape)
# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(partition="dev-clean", root = "/content/11-785-s24-hw1p2/", context=config["context"], limit = None)

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(root = "/content/11-785-s24-hw1p2/", context=config["context"], limit = None)

(1406,)
tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.3837, -0.2862,  5.6537]) tensor(0) torch.Size([2727]) torch.Size([])


In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  50
Input size     :  2727
Output symbols :  42
Train dataset samples = 36091157, batches = 17623
Validation dataset samples = 1928204, batches = 942
Test dataset samples = 1934138, batches = 945


In [ ]:

# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 2727]) torch.Size([2048])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:

# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
# class Network(torch.nn.Module):

class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 512*4),
            torch.nn.BatchNorm1d(512*4),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512*4, 512*4),
            torch.nn.BatchNorm1d(512*4),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512*4, 512*4),
            torch.nn.BatchNorm1d(512*4),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512*4, 512*4),
            torch.nn.BatchNorm1d(512*4),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512*4, 512*4),
            torch.nn.BatchNorm1d(512*4),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512*4, output_size)
        )

    def forward(self, x):
        out = self.model(x)
        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [ ]:

INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 24 million parameters for HW1 (including ensembles)

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0         [2727, 2048]  [2048, 2048]  5.586944M  5.584896M
1_model.BatchNorm1d_1          [2048]  [2048, 2048]     4.096k     2.048k
2_model.GELU_2                      -  [2048, 2048]          -          -
3_model.Dropout_3                   -  [2048, 2048]          -          -
4_model.Linear_4         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
5_model.BatchNorm1d_5          [2048]  [2048, 2048]     4.096k     2.048k
6_model.GELU_6                      -  [2048, 2048]          -          -
7_model.Dropout_7                   -  [2048, 2048]          -          -
8_model.Linear_8         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
9_model.BatchNorm1d_9          [2048]  [2048, 2048]     4.096k     2.048k
10_model.GELU_10                    -  [2048, 2048]          -          -
11_model.Dropout_11                 - 

/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[2727, 2048]","[2048, 2048]",5586944.0,5584896.0
1_model.BatchNorm1d_1,[2048],"[2048, 2048]",4096.0,2048.0
2_model.GELU_2,-,"[2048, 2048]",NaN,NaN
3_model.Dropout_3,-,"[2048, 2048]",NaN,NaN
4_model.Linear_4,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
5_model.BatchNorm1d_5,[2048],"[2048, 2048]",4096.0,2048.0
6_model.GELU_6,-,"[2048, 2048]",NaN,NaN
7_model.Dropout_7,-,"[2048, 2048]",NaN,NaN
8_model.Linear_8,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0


In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr'], weight_decay=0.1) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=1, threshold=1e-4, threshold_mode = 'rel')
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 60,eta_min = 1e-4)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.5)
# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html
scaler = torch.cuda.amp.GradScaler()



# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

36

In [ ]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)
        with torch.cuda.amp.autocast():
          ### Forward Propagation
          logits  = model(frames)

          ### Loss Calculation
          loss    = criterion(logits, phonemes)

        ### Backward Propagation
        scaler.scale(loss).backward()


        ### Gradient Descent
        scaler.step(optimizer)


        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))

        scaler.update()

        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="efb5ba10afc754fad55f8408759058272e35301f") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = f"8-run-dropout_{config['dropout']}-ReduceLROnPlateau", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

wandb: Currently logged in as: jianchengsunny (jiancheng12345). Use `wandb login --relogin` to force relogin


In [ ]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open(f"{run.name}.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save(f"{run.name}.txt")

['/content/wandb/run-20240208_194251-sl70opd0/files/8-run-dropout_0.2-ReduceLROnPlateau.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
best_acc = 0

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    scheduler.step(val_acc)
    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    # updating the model version
    if train_acc > best_acc:
      best_acc = train_acc

      # saving the model and optimizer states
      torch.save({
            'model_state_dict'        : model.state_dict(),
            'optimizer_state_dict'    : optimizer.state_dict()
            }, "Model")

      # creating WandB Artifact
      model_artifact = wandb.Artifact(config['architecture'], type='model')

      # Adding model file to Artifact
      model_artifact.add_file("Model")

      # Saving Artifact to WandB
      run.log_artifact(model_artifact)
### Finish your wandb run
#run.finish()

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim=1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            # TODO
            test_predictions.append(predicted_phonemes)
            #print(predicted_phonemes.shape)
            # print(test_predictions)
            del mfccs, logits
            #torch.cuda.empty_cache()
    all_predictions = torch.cat(test_predictions).cpu().numpy()
    print(all_predictions.shape)
    phoneme_predictions = [PHONEMES[i] for i in all_predictions]
    return phoneme_predictions

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/945 [00:00<?, ?it/s]

(1934138,)


In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

The below section is used for restart the training part stored in wandb, assume you stop the training in the above blocks, you can restart it through wandb.


In [ ]:
# Create your wandb run
run = wandb.init(
    name    = f"8-run-dropout_{config['dropout']}-ReduceLROnPlateau", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    #reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    id     = "sl70opd0", ### Insert specific run id here if you want to resume a previous run
    resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

## Retreiving the model

# Getting the latest version of the artifact
artifact = run.use_artifact('{}:latest'.format(config['architecture']))
# Downloading the artifact
artifact_dir = artifact.download()
# Loading the model
model_dict = torch.load(os.path.join(artifact_dir, 'Model'))


# Loading weights
model.load_state_dict(model_dict['model_state_dict'])
# Loading optimizer state
# optimizer.load_state_dict(model_dict['optimizer_state_dict'])
# change the learning rate

lr,0.0
train_acc,90.83673
train_loss,0.24587
val_acc,85.48343
valid_loss,0.48855


wandb: Downloading large artifact a-model:latest, 257.35MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.6


AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1.0000000000000002e-06
    maximize: False
    weight_decay: 0.01
) <class 'torch.optim.adamw.AdamW'>


In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
best_acc = 0

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    print(f'current learning rate is: {curr_lr}')
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    scheduler.step(val_acc)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    # updating the model version
    if train_acc > best_acc:
      best_acc = train_acc

      # saving the model and optimizer states
      torch.save({
            'model_state_dict'        : model.state_dict(),
            'optimizer_state_dict'    : optimizer.state_dict()
            }, "Model")

      # creating WandB Artifact
      model_artifact = wandb.Artifact(config['architecture'], type='model')

      # Adding model file to Artifact
      model_artifact.add_file("Model")

      # Saving Artifact to WandB
      run.log_artifact(model_artifact)
### Finish your wandb run
#run.finish()


Epoch 1/30
current learning rate is: 5e-06


Train:   0%|          | 0/17623 [00:00<?, ?it/s]

KeyboardInterrupt: 